# **Demo - Interfaccia I2C**

Il notebook mostra come usare $\texttt{Analog Discovery 2}$ per instaurare una connessione seriale di tipo $\texttt{I2C}$.

In questo caso lo script comunica con il sensore $\texttt{MPU-6050}$ e utilizza i seguenti registri:

* `0x6B` Power management
* `0x41` Registro temperatura H
* `0x42` Registro temperatura L

**Interpretazione della lettura**. Il sensore risponde con 6 bytes, i primi 2 bytes codificano la temperatura, nel datasheet è riportato il significato degli altri 4. I due bytes letti vengono entrambe codificati come un `uint16` che chiameremo `utemp`. La conversione finale richiede

        temp = utemp / 340 + 36.53

**Lettura dell'accelerazione/rotazioni'**. può essere implementata in modo analogo, cercando il registro giusto e la conversione corretta

**Qualche accortezza**. Ricordiamo che il sensore fornisce i necessari *pull-up* per il corretto funzionamento dell'interfaccia I2C, quindi è fondamentale che il sensore sia alimentato altrimenti l'avvio del bus I2C darà un errore.

In [3]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import time

# -[MPU6050 - Qualche comando]---------------------------------------------------
PWR_MGMT1 = 0x6B
PWR_MNGMT2 = 0x6C
WHO_AM_I = 0x75
SMPLRT_DIV = 0x19
CONFIG = 0x1A
GYRO_CONFIG = 0x1B
ACCEL_CONFIG = 0x1C
TEMP_OUT_H = [0x41]
TEMP_OUT_L = [0x42]
ACCEL_XOUT_H = [0x3B]
ACCEL_XOUT_L = [0x3C]
ACCEL_YOUT_H = [0x3D]
ACCEL_YOUT_L = [0x3E]
ACCEL_ZOUT_H = [0x3F]
ACCEL_ZOUT_L = [0x40]
GYRO_XOUT_H = [0x43]
GYRO_XOUT_L = [0x44]
GYRO_YOUT_H = [0x45]
GYRO_YOUT_L = [0x46]
GYRO_ZOUT_H = [0x47]
GYRO_ZOUT_L = [0x48]
SAD = 0x68

# -[Configurazione AD2]--------------------------------------------------------
#   1. Connessiene con AD2 e selezione configurazione
ad2 = tdwf.AD2()
#   2. Configurazione alimentazione
try:
    ad2.vdd = 3.3
    ad2.power(True)
    #   3. Impostazione bus I2C
    i2c = tdwf.I2Cbus(ad2.hdwf)  # default 100kHz, SCL = D00, SDA = D01
    devs = i2c.scan()  # verifica cosa è connesso...
    for dev in devs:
        print(f"Device: 0x{dev:02x}")
    #   4. Apre la comunicazione I2C come Master
    sht = tdwf.I2Cdevice(ad2.hdwf,SAD)

    # -[Configurazione sensore]------------------------------------------------------
    sht.write([PWR_MGMT1, 0x80])
    time.sleep(1)
    sht.write([PWR_MGMT1, 0x00])  # sveglia il sensore e seleziona clock, abilita sesore di temperatura
    time.sleep(0.1)
    sht.write([PWR_MNGMT2, 0x00])   # abilita accelerometro e giroscopio
    time.sleep(0.1)
    sht.write([CONFIG, 0x03])       # filtro digitale
    time.sleep(0.1)
    sht.write([SMPLRT_DIV, 0x04])   # sample rate = Gyro output rate / (1 + SMPLRT_DIV)
    time.sleep(0.1)
    sht.write([GYRO_CONFIG, 0x08])  # full scale = +/- 2000 deg/s   SENTIVITY SCALE FACTIO = 65.5 LSB/deg/s
    time.sleep(0.1)
    sht.write([ACCEL_CONFIG, 0x00]) # full scale = +/- 2g   SENTIVITY SCALE FACTIO = 16384 LSB/g
    time.sleep(0.1)

    # -[Funzioni di gestione eventi]-----------------------------------------------

    def on_close(event):
        global flag_run
        flag_run = False

    # def on_key(event):
    #     global flag_run
    #     if event.key == 'escape':  # termina programma
    #         flag_run = False

    # -[Ciclo di misura]-----------------------------------------------------------
    #   1. Creazione figura e link agli eventi
    # fig, ax = plt.subplots(figsize=(12,6))
    # fig.canvas.mpl_connect("close_event", on_close)
    # fig.canvas.mpl_connect("key_press_event", on_key)

    #   2. Inizializzazione variabili - MISURA
    # mytime = []
    # TTv = []
    # flag_first = True
    # flag_run = True
    # ACCEL_X = []
    # ACCEL_Y = []
    # ACCEL_Z = []

    # start_time = time.time()

    #   3. Ciclo di misura
    # while flag_run:
    #     time.sleep(0.1)

    #     sht.writeread([WHO_AM_I], 1)
    #     if sht.vals[0] != 0x68:
    #         print(sht.vals[0])
    #         print("ALARM: Communication lost! Check wires.")
    #         continue # Skip this loop if sensor isn't responding

    #   Misura accelerazione.

        # sht.writeread(ACCEL_XOUT_H,6)
        # ACCEL_X.append((sht.vals[0] << 8 | sht.vals[1])/16384)
        # ACCEL_Y.append((sht.vals[2] << 8 | sht.vals[3])/16384)
        # ACCEL_Z.append((sht.vals[4] << 8 | sht.vals[5])/16384)



    #     sht.writeread(TEMP_OUT_H,2)
    #     TH = sht.vals[0]
    #     #sht.writeread(TEMP_OUT_L,1)
    #     TL = sht.vals[1]

    #      # Complemento a due
    #     TT = (TH << 8) | TL
    #     if TT>=32767:
    #         TT-=65536
    #     TT = TT/340 + 36.53 # Conversione temperatura
    #     TTv.append(TT)
    #     print(f"T = {TT:.2f}C")

    #     mytime.append(time.time()-start_time)
    #     if flag_first:
    #         flag_first = False
    #         hp1, = plt.plot(mytime,TTv, "o", color="tab:orange")
    #         plt.grid(True)
    #         plt.ylabel("Temperature [C]")
    #         plt.xlabel("Time [s]")
    #         plt.title("Press ESC to exit")
    #         plt.show(block=False)
    #         plt.tight_layout()
    #     else:
    #         hp1.set_ydata(TTv)
    #         hp1.set_xdata(mytime)
    #         ax.relim()           
    #         ax.autoscale_view() 
    #         fig.canvas.draw()
    #         fig.canvas.flush_events()
    # # ---------------------------------------
    # plt.close(fig)

    # -[Plot Setup]----------------------------------------------
    # Create a figure with 3 vertical subplots
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 8), sharex=True)
    plt.subplots_adjust(hspace=0.4)

    def on_key(event):
        global flag_run
        if event.key == 'escape':
            flag_run = False

    fig.canvas.mpl_connect("key_press_event", on_key)

    # Data containers
    mytime, ACCEL_X, ACCEL_Y, ACCEL_Z = [], [], [], []
    flag_run = True
    start_time = time.time()

    # Initial plot lines (placeholders)
    lineX, = ax1.plot([], [], marker='o', linestyle='', color='tab:red', label='X-axis')
    lineY, = ax2.plot([], [], marker='o', linestyle='', color='tab:green', label='Y-axis')
    lineZ, = ax3.plot([], [], marker='o', linestyle='', color='tab:blue', label='Z-axis')

    for ax, label in zip([ax1, ax2, ax3], ["X [g]", "Y [g]", "Z [g]"]):
        ax.set_ylabel(label)
        ax.grid(True)
        ax.set_ylim(-2, 2) # Fixed range prevents constant jumping
        ax.legend(loc="upper right")

    ax3.set_xlabel("Time [s]")
    ax1.set_title("Live Acceleration Data (Press ESC to stop) [g]")

    plt.show(block=False)

    # -[Ciclo di misura]------------------------------------------
    while flag_run:
        time.sleep(0.05) # Slightly faster sampling for acceleration
        
        # Read 6 bytes starting from ACCEL_XOUT_H (X_H, X_L, Y_H, Y_L, Z_H, Z_L)
        sht.writeread(ACCEL_XOUT_H, 1)
        ACCEL_X_H = sht.vals[0]
        sht.writeread(ACCEL_XOUT_L, 1)
        ACCEL_X_L = sht.vals[0]
        sht.writeread(ACCEL_YOUT_H, 1)
        ACCEL_Y_H = sht.vals[0]
        sht.writeread(ACCEL_YOUT_L, 1)
        ACCEL_Y_L = sht.vals[0]
        sht.writeread(ACCEL_ZOUT_H, 1)
        ACCEL_Z_H = sht.vals[0]
        sht.writeread(ACCEL_ZOUT_L, 1)
        ACCEL_Z_L = sht.vals[0]
        
        # Helper function for Two's Complement and conversion
        def convert_accel(h, l):
            val = (h << 8) | l
            if val >= 0x8000:
                val -= 0x10000
            return val / 16384.0 # Scale factor for +/- 2g

        current_t = time.time() - start_time
        mytime.append(current_t)
        ACCEL_X.append(convert_accel(ACCEL_X_H, ACCEL_X_L))
        ACCEL_Y.append(convert_accel(ACCEL_Y_H, ACCEL_Y_L))
        ACCEL_Z.append(convert_accel(ACCEL_Z_H, ACCEL_Z_L))

        # Update plots
        lineX.set_data(mytime, ACCEL_X)
        lineY.set_data(mytime, ACCEL_Y)
        lineZ.set_data(mytime, ACCEL_Z)

        # Auto-scaling logic
        for ax in [ax1, ax2, ax3]:
            ax.relim()
            ax.autoscale_view()

        fig.canvas.draw()
        fig.canvas.flush_events()


finally:
    # This block executes even if you stop the kernel or an error occurs
    print("\nCleaning up hardware...")
    plt.close('all')
    ad2.power(False)
    ad2.close()
    print("AD2 Power Off. Process safely terminated.")

Dispositivo #1 [SN:210321B5D8FC, hdwf=1] connesso!
Configurazione #1
Bus I2C pronto...
Device: 0x68

Cleaning up hardware...
Dispositivo disconnesso.
AD2 Power Off. Process safely terminated.
